In [4]:
from dotenv import load_dotenv
import os
import base64
import json
from requests import post,get
import pprint

printer = pprint.PrettyPrinter()

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

### Get Token

In [3]:
def get_token():
    auth_string = client_id+":"+client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers = headers, data = data) # send a post request to the server, return the 
    json_result = json.loads(result.content)
    token = json_result["access_token"]

    return token

def get_auth_header(token):
    return {"Authorization":"Bearer "+token}


### Search for artist

In [78]:
# list out 5 artists
# artist:{"name","popularity","id","external_urls"}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=5" # artist name, type*, limit 

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]

    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    print("The results of searching artist: ")
    for i,artist in enumerate(json_result):
        # popularity = artist["popularity"]
        name = artist["name"]
        print(f"{i+1}. {name}")
    
    return json_result

### Search for track

In [79]:
# list out 5 tracks
# artist:{"name","artists","popularity","id","external_urls"}

def search_for_track(token, track_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={track_name}&type=track&limit=5" # track name, type*, limit 

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)["tracks"]["items"]

    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    
    print("The results of searching track: ")
    for i,track in enumerate(json_result):
        # popularity = track["popularity"]
        name = track["name"]
        artists = ", ".join([ artist["name"] for artist in track["artists"]])
        print(f"{i+1}. {name} - {artists}")
    
    return json_result

### Get recommendations

In [86]:
def get_recommedations(token, seed_artists="", seed_genres="", seed_tracks="",limit=5 ):
    
    query = []  

    if seed_artists:
        query.append(f"seed_artists={seed_artists}")
    if seed_genres:
        query.append(f"seed_genres={seed_genres}")
    if seed_tracks:
        query.append(f"seed_tracks={seed_tracks}")

    query = "&".join(query)

    url = "https://api.spotify.com/v1/recommendations?"+query+f"&limit={limit}"
    headers = get_auth_header(token)
    result = get(url, headers = headers)
    json_result = json.loads(result.content)["tracks"]

    
    print("Your Recommendation:")
    for i,recom in enumerate(json_result):
        track = recom["name"]
        artists = ", ".join([artist["name"] for artist in recom["artists"]])

        print(f"{i+1}. {track} - {artists}")

    return json_result

### Main script

In [84]:
token = get_token()
artists = search_for_artist(token, "Charlie Puth")
artist_select = int(input("Select an artist"))-1 # input
print("------------------------------")
print(f'Your selection is: {artists[artist_select]["name"]} \n')
artist_id = artists[artist_select]["id"]


tracks = search_for_track(token, "See you again")
track_select = int(input("Select a track"))-1 # input
print("------------------------------")
track = tracks[track_select]
name = track["name"]
artists = ", ".join([ artist["name"] for artist in track["artists"]])
print(f'Your selection is: {name} - {artists} \n')
track_id = track["id"]


The results of searching artist: 
1. Charlie Puth
2. Unlike Pluto
3. Charley Pride
4. DNCE
5. Charlie Pittman
------------------------------
Your selection is: Charlie Puth 

The results of searching track: 
1. See You Again (feat. Kali Uchis) - Tyler, The Creator, Kali Uchis
2. See You Again (feat. Charlie Puth) - Wiz Khalifa, Charlie Puth
3. See You Again - Miley Cyrus
4. See You Again - Carrie Underwood
5. See You Again - Wiz Khalifa, Charlie Puth
------------------------------
Your selection is: See You Again - Wiz Khalifa, Charlie Puth /n


In [87]:
recoms = get_recommedations(token=token, seed_artists=artist_id , seed_tracks=track_id)

Your Recommendation:
1. Got Me Good - DNCE
2. Broccoli (feat. Lil Yachty) - DRAM, Lil Yachty
3. Ill Mind of Hopsin 7 - Hopsin
4. Party Girls - Ludacris, Wiz Khalifa, Jeremih, Cashmere Cat
5. Satellites (feat. Wiz Khalifa) - HPG Remix - Kevin Gates, Wiz Khalifa
